<a href="https://colab.research.google.com/github/Mat-O-Lab/MSEO/blob/main/tools/csv_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
#-*- coding: UTF-8 -*-
#@title Code { vertical-output: true, display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import pandas as pd 
import io
import sys
import ast, re
import base64
import json
from dateutil.parser import parse
from contextlib import redirect_stderr
from csv import Sniffer
import chardet 
%matplotlib notebook

!pip install Owlready2
from owlready2 import *

mseo=get_ontology("https://purl.matolab.org/mseo/mid").load()
cco_mu=get_ontology("http://www.ontologyrepository.com/CommonCoreOntologies/Mid/UnitsOfMeasureOntology/").load()
qudt=get_ontology('http://www.qudt.org/qudt/owl/1.0.0/unit.owl').load()

json_ld_context=["http://www.w3.org/ns/csvw", {
    "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/",
    "mseo": mseo.base_iri,
    "xsd": "http://www.w3.org/2001/XMLSchema#"}
    ]

def get_encoding(file_data):
  result = chardet.detect(file_data)
  return result['encoding']

def get_column_separator(file_data):
  file_string = io.StringIO(file_data.decode(encoding.value)) 
  sniffer = Sniffer()
  dialect = sniffer.sniff(file_string.read(256))
  return dialect.delimiter

def get_header_lenght(file_data, separator_string, encoding):
  file_string = io.StringIO(file_data.decode(encoding))  
  f = io.StringIO()
  with redirect_stderr(f):
      df = pd.read_csv(file_string,sep=separator.value,error_bad_lines=False,warn_bad_lines=True,header=None)
  f.seek(0)
  #without utf string code b' 
  warn_str=f.read()[2:-2]
  warnlist=warn_str.split('\\n')[:-1]
  # readout row index and column count in warnings
  line_numbers=[int(re.search('Skipping line (.+?):', line).group(1)) for line in warnlist]
  column_numbers=[int(line[-1]) for line in warnlist]
  column_numbersm1=column_numbers.copy()
  #pop lats element, so column_numbers is always lenght +1
  column_numbersm1.pop(-1)
  #assumes that the file ends with a uniform table with constant column count
  #determine changes in counted columns starting from the last line of file
  changed_column_count_line=[line_numbers[index+1] for index in reversed(range(len(column_numbersm1))) if column_numbersm1[index]!=column_numbers[index+1]]
  #print(changed_column_count_line)
  if changed_column_count_line:
    # additional header has ends in line before the last change of column count
    return changed_column_count_line[0]-1
  else:
    return 0

  # except:
  #   e = sys.exc_info()[1]
  #   if 'Error tokenizing' in e.args[0]:
  #     #example Error tokenizing data. C error: Expected 3 fields in line 17, saw 5
  #     # column header will be at line 17
  #     line = int(re.search('fields in line (.+?),', e.args[0]).group(1))-1
  #     return line
  # # return zero if ther is no error -> no additional header
  return 0

def get_num_header_rows_and_dataframe(file_data,separator_string, header_lenght, encoding):
  file_string = io.StringIO(file_data.decode(encoding))
  num_header_rows=1
  table_data = pd.read_csv(file_string,header=list(range(num_header_rows)),sep=separator_string,skiprows=header_lenght,encoding=encoding)
  #print(table_data.columns,header_lenght)
  #print(all(table_data[column].dtype=='object' for column in table_data.columns),num_header_rows)
  while all(table_data[column].dtype=='object' for column in table_data.columns):
    #print(all(table_data[column].dtype=='object' for column in table_data.columns),num_header_rows)
    #has probablly multi line header
    num_header_rows+=1
    file_string = io.StringIO(file_data.decode(encoding))  
    #print(num_header_rows)
    table_data = pd.read_csv(file_string,header=list(range(num_header_rows)),sep=separator_string,skiprows=header_lenght,encoding=encoding)
    if num_header_rows>10:
      # something is wrong - returning zero
      return 0
  return num_header_rows, table_data

def get_unit(string):
  found=list(cco_mu.search(alternative_label=string))\
          +list(cco_mu.search(SI_unit_symbol=string))\
          +list(mseo.search(alternative_label=string))\
          +list(mseo.search(SI_unit_symbol=string))\
          +list(qudt.search(symbol=string))\
          +list(qudt.search(abbreviation=string))\
          +list(qudt.search(ucumCode=string))
  if found:
    return {"cco:uses_measurement_unit": {"@id": str(found[0].iri), "@type": str(found[0].is_a)}}
  elif pd.isna(string):
    return {}
  else:
    return {"unitString": string}

def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

def get_value_type(str):
    str=str.strip()
    if len(str) == 0: return 'BLANK'
    try:
        t=ast.literal_eval(str)
    except ValueError:
        return 'TEXT'
    except SyntaxError:
        if is_date(str):
          return 'DATE'
        else:
          return 'TEXT'
    else:
        if type(t) in [int, float, bool]:
          if type(t) is int:
              return 'INT'
          if t in set((True,False)):
              return 'BOOL'
          if type(t) is float:
              return 'FLOAT'
        else:
            return 'TEXT' 

def describe_value(value_string):
  if get_value_type(value_string)=='INT':
    return {'cco:has_interger_value': {'@value':value_string, '@type': 'xsd:integer'}}
  elif get_value_type(value_string)=='BOOL':
    return {'cco:has_bolean_value': {'@value':value_string, '@type': 'xsd:boolean'}}
  elif get_value_type(value_string)=='FLOAT':
    return {'cco:has_decimal_value': {'@value':value_string, '@type': 'xsd:decimal'}}
  elif get_value_type(value_string)=='DATE':
    return {'cco:has_datetime_value': {'@value':str(parse(value_string)), '@type': 'xsd:dateTime'}}  
  else:
    return {'cco:has_text_value': {'@value':value_string, '@type': 'xsd:string'}}

umlaute_dict = {
    '\u00e4': 'ae',  # U+00E4	   \xc3\xa4
    '\u00f6': 'oe',  # U+00F6	   \xc3\xb6
    '\u00fc': 'ue',  # U+00FC	   \xc3\xbc
    '\u00c4': 'Ae',  # U+00C4	   \xc3\x84
    '\u00d6': 'Oe',  # U+00D6	   \xc3\x96
    '\u00dc': 'Ue',  # U+00DC	   \xc3\x9c
    '\u00df': 'ss',  # U+00DF	   \xc3\x9f
}

def make_id(string):
  for k in umlaute_dict.keys():
        string = string.replace(k, umlaute_dict[k])
  return '_:'+re.sub('[^A-ZÜÖÄa-z0-9]+', '', string.title().replace(" ", ""))

def process_file(file_name,file_data,separator,encoding):
  #init results dict
  metadata_csvw = dict()
  metadata_csvw["@context"]=json_ld_context
  metadata_csvw["url"]=file_name
  # get lenght of additional header
  header_lenght=get_header_lenght(file_data,separator,encoding)
  #print(header_lenght)
  # read additional header lines and provide as meta in results dict
  file_string = io.StringIO(file_data.decode(encoding))
  header_data = pd.read_csv(file_string,names=['param','value','unit'],header=None,sep=separator,nrows=header_lenght,encoding=encoding)
  header_data['row']=header_data.index
  header_data.set_index('param',inplace=True)
  #print(header_data)
  #metadata_csvw["params"]=header_data.dropna().to_dict(orient='index')
  param_json=list()
  info_line_iri="cco:InformationLine"
  for parm_name, data in header_data.to_dict(orient='index').items():
    describe_value(data['value'])
    json_str={**{'@id': make_id(parm_name),
                 'label':parm_name,'@type': info_line_iri,
                 'mseo:has_row_index': {
                     "@value": data['row'], 
                     "@type": "xsd:integer"}
                 },
              **describe_value(data['value']),            
              **get_unit(data['unit'])}
    #print(json_str)
    param_json.append(json_str)
  metadata_csvw["params"]=param_json
  # read tabular data structure, and determine number of header lines for column description used
  header_lines, table_data=get_num_header_rows_and_dataframe(file_data,separator,header_lenght,encoding)
  # describe dialect
  metadata_csvw["dialect"]={"delimiter": separator,
  "skipRows": header_lenght, "headerRowCount": header_lines, "encoding": encoding}
  # describe columns
  if header_lines==1:
    metadata_csvw["tableSchema"]={"columns":list({'titles':column} for column in table_data.columns)}
  else:
    column_json=list()
    for index, (title,unit_str) in enumerate(table_data.columns):
      json_str={**{'titles': title},**get_unit(unit_str)}
      #print(json_str)
      column_json.append(json_str)
    metadata_csvw["tableSchema"]={"columns":column_json}
  result=json.dumps(metadata_csvw, indent = 4)
  meta_file_name = file_name.split(sep='.')[0] + '-metadata.json'
  return meta_file_name, result


In [118]:
#@title Dialog { vertical-output: true }
# dialog
uploader = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description='Upload'

)
file= widgets.HBox([widgets.Label(value="File:"), uploader])
encoding = widgets.Dropdown(
    options=['auto', 'ISO-8859-1', 'UTF-8', 'ascii', 'latin-1','cp273'],
    value='auto',
    description='Encoding:',
    disabled=False,
)
separator = widgets.Dropdown(
    options=['auto', ';', '\t', '|'],
    value='auto',
    description='separator:',
    disabled=False,
)
settings= widgets.HBox([encoding, separator])
button = widgets.Button(description='Process!', layout=widgets.Layout(width='200px')); 
out = widgets.Output()

def on_button_clicked(_):
  # "linking function with output"
  with out:
  # what happens when we press the button
    clear_output()
    if not uploader.value.keys():
      print('pls upload a file first')
      return
    input_file=uploader.value[list(uploader.value.keys())[0]]
    file_name = input_file['metadata']['name']
    file_data = input_file['content']
    if encoding.value=='auto':
      encoding.value=get_encoding(file_data)
    if separator.value=='auto':
      separator.value=get_column_separator(file_data)
    metafile_name, result =process_file(file_name,file_data,separator.value,encoding.value)
    print(result)
    res = result
    b64 = base64.b64encode(res.encode())
    payload = b64.decode()
    html_buttons = '''<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="{filename}" href="data:text/json;base64,{payload}" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
    </a>
    </body>
    </html>
    '''
    html_button = html_buttons.format(payload=payload,filename=metafile_name)
    display(widgets.HTML(html_button))

button.on_click(on_button_clicked)
process = widgets.VBox([button,out])
display(file,settings,process)